In [ ]:
!pip install datasets py7zr

In [4]:
# Important: install these dependencies first:
# pip install datasets py7zr
from datasets import load_dataset
import os
import pandas as pd

In [5]:
os.environ["REPLICATE_API_TOKEN"] = "r8_XxUNPsB5I1iEJXo72xT2bhxqasc8oaQ3t1sOd"


In [6]:
df = pd.read_parquet("data/st_trainining_input.parquet")

In [37]:
PROMPT_TEMPLATE = "[INST] <<SYS>>\nExtract attributes from the given e-commerce customer query.\nPossible attributes are 'product type', 'brand', 'gender', 'color', 'size'.<</SYS>>\n\nInput:\n{query} [/INST]\n\nOutput: {output}"


In [38]:
print(PROMPT_TEMPLATE)

[INST] <<SYS>>
Extract attributes from the given e-commerce customer query.
Possible attributes are 'product type', 'brand', 'gender', 'color', 'size'.<</SYS>>

Input:
{query} [/INST]

Output: {output}


In [39]:
print(PROMPT_TEMPLATE)

[INST] <<SYS>>
Extract attributes from the given e-commerce customer query.
Possible attributes are 'product type', 'brand', 'gender', 'color', 'size'.<</SYS>>

Input:
{query} [/INST]

Output: {output}


In [40]:
def format_instruction(sample):
    return PROMPT_TEMPLATE.format(query=sample["query"], output=sample["output"])


In [41]:
df['text'] = df.apply(format_instruction , axis=1)

In [42]:
df.iloc[0].to_dict()

{'query': ' revent 80 cfm',
 'output': '{"brand":null,"gender":"not_given","product_type":null,"color":null,"size":"80 cfm"}',
 'text': '[INST] <<SYS>>\nExtract attributes from the given e-commerce customer query.\nPossible attributes are \'product type\', \'brand\', \'gender\', \'color\', \'size\'.<</SYS>>\n\nInput:\n revent 80 cfm [/INST]\n\nOutput: {"brand":null,"gender":"not_given","product_type":null,"color":null,"size":"80 cfm"}'}

In [50]:
print(df.iloc[0].to_dict()['text'])

[INST] <<SYS>>
Extract attributes from the given e-commerce customer query.
Possible attributes are 'product type', 'brand', 'gender', 'color', 'size'.<</SYS>>

Input:
 revent 80 cfm [/INST]

Output: {"brand":null,"gender":"not_given","product_type":null,"color":null,"size":"80 cfm"}


In [43]:
df[['text']].to_json("data/st_train.jsonl",lines=True, orient="records")

In [44]:
!head data/st_train.jsonl -n 5

{"text":"[INST] <<SYS>>\nExtract attributes from the given e-commerce customer query.\nPossible attributes are 'product type', 'brand', 'gender', 'color', 'size'.<<\/SYS>>\n\nInput:\n revent 80 cfm [\/INST]\n\nOutput: {\"brand\":null,\"gender\":\"not_given\",\"product_type\":null,\"color\":null,\"size\":\"80 cfm\"}"}
{"text":"[INST] <<SYS>>\nExtract attributes from the given e-commerce customer query.\nPossible attributes are 'product type', 'brand', 'gender', 'color', 'size'.<<\/SYS>>\n\nInput:\nbathroom fan without light [\/INST]\n\nOutput: {\"brand\":null,\"gender\":\"not_given\",\"product_type\":\"bathroom fan\",\"color\":null,\"size\":null}"}
{"text":"[INST] <<SYS>>\nExtract attributes from the given e-commerce customer query.\nPossible attributes are 'product type', 'brand', 'gender', 'color', 'size'.<<\/SYS>>\n\nInput:\nbathroom fan with light [\/INST]\n\nOutput: {\"brand\":null,\"gender\":\"not_given\",\"product_type\":\"bathroom fan\",\"color\":null,\"size\":null}"}
{"text":"[

In [45]:
%%bash

export REPLICATE_API_TOKEN="r8_QpQZBY0Jc4rJDxfNt3CRqaJgAQIzfOP2frFLj"

RESPONSE=$(curl -s -X POST -H "Authorization: Token $REPLICATE_API_TOKEN" https://dreambooth-api-experimental.replicate.com/v1/upload/data.jsonl)

curl -X PUT -H "Content-Type: application/jsonl" --upload-file data/st_train.jsonl "$(jq -r ".upload_url" <<< "$RESPONSE")"

SERVING_URL=$(jq -r ".serving_url" <<< $RESPONSE)
echo $SERVING_URL

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78367    0     0  100 78367      0  15064  0:00:05  0:00:05 --:--:-- 19212


https://replicate.delivery/pbxt/JyrgXaadFpOt1V3J6CGRW1Jdoe6t2DObqdYjm9aFG6MIcESm/data.jsonl


In [ ]:
!ls

In [33]:
import replicate

In [ ]:
??replicate.trainings.create

In [53]:

model_version = "meta/llama-2-7b:73001d654114dad81ec65da3b834e2f691af1e1526453189b7bf36fb3f32d0f9"
model_version="meta/llama-2-7b-chat:13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0"

In [54]:


training = replicate.trainings.create(
  version=model_version,
  input={
    "train_data": "https://replicate.delivery/pbxt/JyrgXaadFpOt1V3J6CGRW1Jdoe6t2DObqdYjm9aFG6MIcESm/data.jsonl",
    "num_train_epochs": 3
  },
  destination="npatta01/llama2-ecommerce-st"
)

print(training)

id='y4czartb25qvhmjuqt3cw5elyi' model='meta/llama-2-7b-chat' version='13c3cdee13ee059ab779f0291d29054dab00a47dad8261375654de5540165fb0' destination=None status='starting' input={'num_train_epochs': 3, 'train_data': 'https://replicate.delivery/pbxt/JyrgXaadFpOt1V3J6CGRW1Jdoe6t2DObqdYjm9aFG6MIcESm/data.jsonl'} output=None logs='' error=None created_at='2023-12-02T21:52:22.374751149Z' started_at=None completed_at=None urls={'cancel': 'https://api.replicate.com/v1/predictions/y4czartb25qvhmjuqt3cw5elyi/cancel', 'get': 'https://api.replicate.com/v1/predictions/y4czartb25qvhmjuqt3cw5elyi'}


In [56]:
# If you've got a handle to the object returned by create()
training.reload()

# If you've got the training ID
training = replicate.trainings.get("y4czartb25qvhmjuqt3cw5elyi")

if training.status == "succeeded":
    print(training.output)
    # {"weights": "...", "version": "..."